In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, recall_score, classification_report

In [10]:
digits = load_digits()
X, y = digits.data, digits.target
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42, stratify=y_trainval)

models = {
    "Logistic Regression": (LogisticRegression(max_iter=10000), {"C": [0.01, 0.1, 1, 10, 100]}),
    "Decision Tree": (DecisionTreeClassifier(), {"max_depth": [3, 5, 10, 20, None]}),
    "K-Nearest Neighbors": (KNeighborsClassifier(), {"n_neighbors": [3, 5, 7, 9]}),
    "Neural Network": (MLPClassifier(max_iter=1000), {"hidden_layer_sizes": [(50,), (100,), (100, 50)], "alpha": [0.0001, 0.001, 0.01]})
}

results = []

for name, (model, params) in models.items():
    gs = GridSearchCV(model, params, cv=3, scoring='accuracy')
    gs.fit(X_train, y_train)
    best_model = gs.best_estimator_

    y_pred = best_model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    recall_per_class = recall_score(y_test, y_pred, average=None)
    best_params = gs.best_params_

    acc_rounded = round(acc * 100, 2)
    recall_per_class_rounded = np.round(recall_per_class * 100, 2)
    results.append({
        "Model": name,
        "Best Params": best_params,
        "Accuracy": acc_rounded,
        "Recall per class": recall_per_class_rounded
    })

df_results = pd.DataFrame(results)
print(df_results)
df_results.to_csv("model_comparison_results.csv", index=False)


                 Model                                       Best Params  \
0  Logistic Regression                                        {'C': 0.1}   
1        Decision Tree                                 {'max_depth': 20}   
2  K-Nearest Neighbors                                {'n_neighbors': 3}   
3       Neural Network  {'alpha': 0.01, 'hidden_layer_sizes': (100, 50)}   

   Accuracy                                   Recall per class  
0     95.83  [97.22, 88.89, 100.0, 100.0, 100.0, 100.0, 97....  
1     83.06  [94.44, 72.22, 82.86, 81.08, 86.11, 91.89, 86....  
2     98.61  [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...  
3     96.67  [100.0, 91.67, 100.0, 97.3, 100.0, 100.0, 97.2...  
